In [1]:
import pandas as pd 
import numpy as np 
import glob
from tqdm import tqdm
from ScrimLog import *
from PeriEventTimeHistogram import *

In [2]:
# FinalStat
# scrim_csv = ScrimLog().update_FinalStat_to_csv()
scrim_sql = ScrimLog().update_FinalStat_to_sql()

File Exported to scrim_finalstat: 20210414_01_O2_Blizzard World.csv
File Exported to scrim_finalstat: 20210414_01_O2_Dorado.csv


In [3]:
# 'Shift' Skill
cooldown1 = PETH()
cooldown1.set_search_condition(event_name='Cooldown1%/s', threshold=0.01)
cooldown1.update_PETH_to_sql()

# 'E' Skill
cooldown2 = PETH()
cooldown2.set_search_condition(event_name='Cooldown2%/s', threshold=0.01)
cooldown2.update_PETH_to_sql()

# 'Q' Skill
UU = PETH()
UU.set_search_condition(event_name='UltimatesUsed/s', threshold=1)
UU.update_PETH_to_sql()

# 'R click' Skill
cooldownSecondaryFire = PETH()
cooldownSecondaryFire.set_search_condition(event_name='CooldownSecondaryFire%/s', threshold=0.01)
cooldownSecondaryFire.update_PETH_to_sql()

# 'Ctrl' Skill
cooldownCrouching = PETH()
cooldownCrouching.set_search_condition(event_name='CooldownCrouching%/s', threshold=0.01)
cooldownCrouching.update_PETH_to_sql()

# 'FB'
FB = PETH()
FB.set_search_condition(event_name='FinalBlows/s', threshold=1)
FB.update_PETH_to_sql()

File Exported to scrim_peth: 20210414_01_o2_blizzard world_CD1%
File Exported to scrim_peth: 20210414_01_o2_dorado_CD1%
File Exported to scrim_peth: 20210414_01_o2_blizzard world_CD2%
File Exported to scrim_peth: 20210414_01_o2_dorado_CD2%
File Exported to scrim_peth: 20210414_01_o2_blizzard world_UltU
File Exported to scrim_peth: 20210414_01_o2_dorado_UltU
File Exported to scrim_peth: 20210414_01_o2_blizzard world_CD2nd%
File Exported to scrim_peth: 20210414_01_o2_dorado_CD2nd%
File Exported to scrim_peth: 20210414_01_o2_blizzard world_CDCtrl%
File Exported to scrim_peth: 20210414_01_o2_dorado_CDCtrl%
File Exported to scrim_peth: 20210414_01_o2_blizzard world_FB
File Exported to scrim_peth: 20210414_01_o2_dorado_FB
